# Imports and inits

In [1]:
'''Installations'''

!pip install evaluate
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simp

In [2]:
import numpy as np
import pandas as pd
import evaluate
import transformers
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, TextClassificationPipeline 

In [3]:
'''For Google colab '''

# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path = 'wantwords&opted/wantwords&opted.csv'

# Data init

In [5]:
data = pd.read_csv(path, dtype="string")
data['Definition'] = data['Definition'].astype(str)

In [6]:
data = data[:100]

In [7]:
'''Convert classes to numbers'''
word_dict = {} 
i = 0
for w in data['Word'].unique():
    word_dict[w] = i
    i += 1

'''Convert numbers back to words'''
idx2word = {v:k for k,v in word_dict.items()}

In [8]:
df_train, df_test = train_test_split(data[['Definition','Word']], test_size=0.2)

# Model and tokenizer init

In [9]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli', num_labels = len(word_dict), ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([10, 1024]) in the model instantiated
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train and test dataset preperation

In [10]:
train_enc = tokenizer(df_train['Definition'].to_list(), truncation=True, padding=True)


In [11]:
test_enc = tokenizer(df_test['Definition'].to_list(), truncation=True, padding=True)

In [12]:
df_train['Word'] = df_train['Word'].apply(lambda x: word_dict[x])

In [13]:
df_train['Word'].unique()

array([0, 4, 9, 5, 2, 3, 8, 7, 6, 1])

In [14]:
class RevDictDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        self.labels = self.labels.to_list()
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [15]:
'''One hot encoding of classes'''
label_enum = {k:j for j, k in enumerate(df_train['Word'].unique())}
num_labels = len(label_enum)
df_train['labels'] = df_train['Word'].apply(lambda x: [1.0 if label_enum[x]==i else 0.0 for i in range(num_labels)])

In [16]:
train_dataset = RevDictDataset(train_enc, df_train['labels'])

In [17]:
df_train['labels'].iloc[1]

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# Model training

In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)
nli_model.config.problem_type = 'multi_label_classification'

In [19]:
trainer = Trainer(
    model=nli_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset           # evaluation dataset
)


In [20]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,0.706800
20,0.629300
30,0.523100
40,0.384300
50,0.283200
60,0.273900


TrainOutput(global_step=60, training_loss=0.4667580445607503, metrics={'train_runtime': 24.0636, 'train_samples_per_second': 9.974, 'train_steps_per_second': 2.493, 'total_flos': 29038155531840.0, 'train_loss': 0.4667580445607503, 'epoch': 3.0})

# Model Inference

In [26]:
pipe = TextClassificationPipeline(model=nli_model, tokenizer=tokenizer, return_all_scores=True, device=0)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [27]:
'''Inference on a single sentence'''

output = pipe(['too much'])

In [28]:
scores = [a['score'] for a in output[0]]

In [29]:
pred_scores = [(scores[i] ,idx2word[i]) for i in range(len(scores))]
pred_scores.sort(reverse=True)
pred_scores

[(0.44116687774658203, 'pardoned'),
 (0.3226940929889679, 'berried'),
 (0.21313296258449554, 'sodomize'),
 (0.20543833076953888, 'excess'),
 (0.16950136423110962, 'roadrunner'),
 (0.154733806848526, 'trichromatic'),
 (0.13076969981193542, 'deaminase'),
 (0.11031726747751236, 'toppings'),
 (0.08632007241249084, 'fingerpicking'),
 (0.061041492968797684, 'luxuriousness')]